In [1]:
import numpy as np 
import pandas as pd 
import re
import nltk
from pyvi import ViTokenizer 

from flask import Flask, render_template, request, redirect

In [2]:
app= Flask(__name__,template_folder='../template')

In [3]:
data_source_url = 'C:/Users/justf/desktop/comment.csv'
df = pd.read_csv(data_source_url)
comments = df.iloc[:, 0].values
labels = df.iloc[:,4].values

In [4]:
from string import punctuation
stop_word = []
f= open("C:/Users/justf/Desktop/stop_word.txt",encoding="utf-8")
text = f.read()

In [5]:
for word in text.split() :
      stop_word.append(word)
f.close()

In [6]:
punc = list(punctuation)
stop_word = stop_word + punc

In [7]:
processed_comments = []

for sentence in range(0, len(comments)):
    # Remove all the special characters
    processed_comment = re.sub(r'\W', ' ', str(comments[sentence]))

    # remove all single characters
    processed_comment= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_comment)

    # Remove single characters from the start
    processed_comment = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_comment) 

    # Substituting multiple spaces with single space
    processed_comment = re.sub(r'\s+', ' ', processed_comment, flags=re.I)

    # Removing prefixed 'b'
    processed_comment = re.sub(r'^b\s+', '', processed_comment)

    # Converting to Lowercase
    processed_comment = processed_comment.lower()

    processed_comments.append(processed_comment)

In [8]:
sentences = []
for commenthuy in processed_comments:
    sent = []
    for word in commenthuy.split(" ") :
        if (word not in stop_word) :
            if ("_" in word) or (word.isalpha() == True):
                sent.append(word)
    sentences.append(" ".join(sent))

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer (max_features=3900, min_df=5, max_df=0.8)

In [10]:
vectorizer.fit(sentences)
X = vectorizer.transform(sentences).toarray()

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=0)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
model = RandomForestClassifier()
model.fit(X_train,y_train)
y_pre = model.predict(X_test)
print(classification_report(y_test,y_pre))
print(accuracy_score(y_test, y_pre))

              precision    recall  f1-score   support

           n       0.79      0.78      0.78       587
           p       0.78      0.79      0.78       583

    accuracy                           0.78      1170
   macro avg       0.78      0.78      0.78      1170
weighted avg       0.78      0.78      0.78      1170

0.782051282051282


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
model1 = LogisticRegression()
model1.fit(X_train,y_train)
y_pre1 = model1.predict(X_test)
print(classification_report(y_test,y_pre1))
print(accuracy_score(y_test, y_pre1))

              precision    recall  f1-score   support

           n       0.79      0.82      0.81       587
           p       0.81      0.79      0.80       583

    accuracy                           0.80      1170
   macro avg       0.80      0.80      0.80      1170
weighted avg       0.80      0.80      0.80      1170

0.8025641025641026


In [14]:
text2=["Thầy Thiện lung linh quá ","Máy chạy giật lag","Ngon quá"]
text2_token = []
for sentence1 in text2:
     # Remove all the special characters
    processed_text2 = re.sub(r'\W', ' ', str(sentence1))

    # remove all single characters
    processed_text2= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_text2)

    # Remove single characters from the start
    processed_text2 = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_text2) 

    # Substituting multiple spaces with single space
    processed_text2 = re.sub(r'\s+', ' ', processed_text2, flags=re.I)

    # Removing prefixed 'b'
    processed_text2 = re.sub(r'^b\s+', '', processed_text2)

    # Converting to Lowercase
    processed_text2 = processed_text2.lower()
    test_token = ViTokenizer.tokenize(processed_text2)
    text2_token.append(test_token)
print(text2_token)

['thầy thiện lung_linh quá', 'máy chạy giật lag', 'ngon quá']


In [15]:
sentences_text2 = []
for commenthuy1 in text2_token:
    sent1 = []
    for word1 in commenthuy1.split(" ") :
        if (word1 not in stop_word) :
            if ("_" in word1) or (word1.isalpha() == True):
                sent1.append(word1)
    sentences_text2.append(" ".join(sent1))
print(sentences_text2)

['thầy thiện lung_linh quá', 'máy chạy giật lag', 'ngon quá']


In [16]:
test1 = vectorizer.transform(sentences_text2)
print(model.predict(test1))

['p' 'n' 'p']


In [17]:
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        sentences = []
        sentence = request.form['sentiment']
        sentences.append(sentence)
        text3_token = []
        for sentence2 in sentences:
            processed_text3 = (sentence2)
            processed_text3= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_text3)
            processed_text3 = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_text3)
            processed_text3 = re.sub(r'\s+', ' ', processed_text3, flags=re.I)
            processed_text3 = re.sub(r'^b\s+', '', processed_text3)
            processed_text3 = processed_text3.lower()
            test_token2 = ViTokenizer.tokenize(processed_text3)
            text3_token.append(test_token2)
        sentences_text3 = []
        for commenthuy2 in text3_token:
            sent3 = []
            for word3 in commenthuy2.split(" ") :
                if (word3 not in stop_word) :
                    if ("_" in word3) or (word3.isalpha() == True):
                        sent3.append(word3)
            sentences_text3.append(" ".join(sent3))
        sentence_vector = vectorizer.transform(sentences_text3)
        output = model.predict(sentence_vector)
        result = None
        if str(output) == '[\'n\']':
            result = 'Negative'
        if str(output) == '[\'p\']':
            result = 'Positive'
        if str(output) == '[\'p\']' and ("không" in output)==true:
            result = 'Negative'
        return render_template('sentiment.html', output=result)
    return render_template('sentiment.html')

In [24]:
from sklearn.svm import SVC
model2 = SVC(kernel='rbf')
model2.fit(X_train,y_train)
y_pre2 = model2.predict(X_test)
print(classification_report(y_test,y_pre2))
print(accuracy_score(y_test, y_pre2))

              precision    recall  f1-score   support

           n       0.79      0.83      0.81       587
           p       0.82      0.78      0.80       583

    accuracy                           0.80      1170
   macro avg       0.80      0.80      0.80      1170
weighted avg       0.80      0.80      0.80      1170

0.8034188034188035


In [19]:
from sklearn.neighbors import KNeighborsClassifier
model3 = KNeighborsClassifier(n_neighbors=5)
model3.fit(X_train,y_train)
y_pre3 = model3.predict(X_test)
print(classification_report(y_test,y_pre3))
print(accuracy_score(y_test, y_pre3))

              precision    recall  f1-score   support

           n       0.82      0.05      0.10       587
           p       0.51      0.99      0.67       583

    accuracy                           0.52      1170
   macro avg       0.66      0.52      0.39      1170
weighted avg       0.66      0.52      0.38      1170

0.5188034188034188
